# **Text, Web, & Media Analytics Assignment 2**

# Setup

In [1]:
# NOTE: ORDER ME PLEASE 🙇‍♂️
import os
import math

# import sklearn
# import matplotlib.pyplot as plt
# import numpy as np

from ir_models import BM25, JM_LM
from ir_tools import write_scores_to_file, term_specificity
from parsing_functions import parse_stop_words, parse_collection, parse_query, parse_query_set, parse_evaluations

from data_structures import bow_document_collection

In [2]:
# Parse in stop words
stop_words = parse_stop_words('common-english-words.txt')

# Load the document set (series of collection objects)
document_set = {}
input_path = 'Data_Collection'
for collection_path in os.listdir(input_path):
    data_key = collection_path.split('_C', 1)[1]
    document_set[data_key] = parse_collection(stop_words, os.path.join(input_path, collection_path))

# Parse in evaluation benchmarks
evaluations = parse_evaluations('EvaluationBenchmark/')

# Parse in query set, apply term specificity to parsed queries
query_frame = parse_query_set('the50Queries.txt')
query_frame['parsed_titles'] = query_frame['title'].apply(lambda row: parse_query(row, stop_words))
query_frame['weighted_terms'] =\
    query_frame.apply(lambda row: 
        term_specificity(
            document_set[row['number']], row['parsed_titles'], 
            evaluations[row['number']], 0.0, 0.5
            ), 
        axis = 1)

# Task 1: BM25

In [17]:
BM25_results = {}

for query_key, collection in document_set.items():
    query = query_frame.loc[query_frame['number'] == query_key, 'weighted_terms'].iloc[0]
    BM25_results[query_key] = BM25(collection, query)
    write_scores_to_file(BM25_results[query_key], f"BM25_R{query_key}Ranking")

# Task 2: Jelinek-Mercer Language Model

In [55]:
JM_LM_results = {}

for query_key, collection in document_set.items():
    query = query_frame.loc[query_frame['number'] == query_key, 'weighted_terms'].iloc[0]
    JM_LM_results[query_key] = JM_LM(collection, query)
    write_scores_to_file(JM_LM_results[query_key], f"JM_LM_R{query_key}Ranking")    

# Task 3: Pseudo-Relevance Model

In [15]:
def cosine_similarity(vector1: list, vector2: list) -> float:
    """
    This function calculates the cosine similarity for two vectors.

    Cosine similarity is a measure of similarity between two non-zero vectors
    of an inner product space that measures the cosine of the angle between them.
    The cosine similarity is particularly used in positive spaces where the
    outcome is neatly bounded in [0,1].
    """

    # Calculate the dot product of the two vectors; sum of the products of the corresponding elements of the vectors
    dot_product = sum(x * y for x, y in zip(vector1, vector2))

    # Compute the Euclidean norms (also called the magnitude or length); square root of the sum of the squares of the elements
    norm1 = math.sqrt(sum(x ** 2 for x in vector1))
    norm2 = math.sqrt(sum(x ** 2 for x in vector2))

    # Compute the cosine similarity; dot product of the vectors divided by the product of their norms.
    return dot_product / (norm1 * norm2) if (norm1 and norm2) != 0 else 0.0  # return similarity, handling 0 case for a Euclidean norm of 0

def Vector_Space_Model(collection: bow_document_collection, query: dict) -> dict:
    """
    The Vector Space Model is an algorithm that calculates scores for each document
    based on a given query. The model incorporates certain dimensions that can be
    changed appropriately, such as similiarity measures, query and the equation for 
    calculating document weights. The following function uses the cosine similarity 
    and the tf-idf equation. It returns a dictionary that contains documents and 
    scores respectively.
    """

    # Type check to ensure coll is a bow_document_collection
    if not isinstance(collection, bow_document_collection):
        raise TypeError("collection: must be a bow_document_collection object.")
    
    # If collection contains no documents, raise attribute error
    if len(collection.docs) == 0:
        raise AttributeError("collection: object contains no documents (bow_document objects).")
    
    # Type check to ensure query is a dict
    if not isinstance(query, dict):
        raise TypeError("query: must be a dict object.")
    
    # Initializations
    document_similarity_scores = {}
    document_vectors = {}
    idf_component = {}

    collection_term_frequency = collection.term_frequency  # dict of the total number of times each term appears in collection
    N = len(collection.docs)  # total number of documents in the collection

    # Calculate idf for all terms
    for term, doc_freq in collection_term_frequency.items():
        idf_component[term] = math.log(N / doc_freq, 10)

    # Get all terms of the collection
    all_terms = set(collection_term_frequency.keys())

    # Represents a general n-dimensional vector, which will be used to construct
    # the query- and document-vector. Therefore, a normalization is not necessary, 
    # due to the fact that the algorithm doesn't compare documents and different queries against each other.
    vector = [0] * len(all_terms)
    query_vector = [0] * len(all_terms)

    # Convert the query parameter into a vector representation
    for index, term in enumerate(all_terms):
        query_vector[index] = query.get(term, 0)
    
    # Calculate weights for each document
    for doc_ID, doc in collection.docs.items():
        term_freq_dict = doc.get_term_list()

        # Loop through all terms
        for index, term in enumerate(all_terms):
            # Calculate the term weigths using tf-idf equation
            vector[index] = term_freq_dict.get(term, 0) * idf_component.get(term, 0)
        
        # Store weighted vector for every document
        document_vectors[doc_ID] = vector 

    # Calculate cosine similarity for query and documents
    for doc_ID, doc_vector in document_vectors.items():
        document_similarity_scores[doc_ID] = cosine_similarity(doc_vector, query_vector)

    # Return documents in descending order (based on values of the weights)
    return dict(sorted(document_similarity_scores.items(), key=lambda item: item[1], reverse=True))


In [16]:
def w5(collection: bow_document_collection, marked_documents, theta) -> dict:
    """
    This function calculates the weight-5 score for the whole collection based 
    on a dictionary that contains relevant and irrelevant documents. Theta is 
    a parameter which can be changed due to receive reasonable results.
    This function returns a dictionary with features and their calculated 
    weights.
    """
    # Type check to ensure coll is a bow_document_collection
    if not isinstance(collection, bow_document_collection):
        raise TypeError("collection: must be a bow_document_collection object.")
    
    # If no collection contains no documents, raise attribute error
    if len(collection.docs) == 0:
        raise AttributeError("Rcv1collection: object contains no documents (Rcv1Doc objects).")
    
    # Type check to ensure query is a dict
    if not isinstance(query, dict):
        raise TypeError("query: must be a dict object.")
    
    # Initializations
    T = {}
    ntk = {}
    R = 0
    N = len(collection.docs)  # total number of documents in the collection
    meanW5 = 0
    
    # Select T from positive documents and r(tk) + n(tk) + R
    for doc_id, doc in collection.docs.items():
        for term, frequency in doc.get_term_list().items():
            if marked_documents[doc_id] == 1:
                T[term] = T.get(term, 0) + 1
            ntk[term] = ntk.get(term, 0) + 1

    # Calculates the amount of relevent documents
    for id, marker in  marked_documents.items():
        if marker == 1:
            R += marker    

    # Calculates the w5-score for each term
    for term, rtk in T.items():
        T[term] = ((rtk + 0.5) / (R-rtk + 0.5)) / ((ntk[term]-rtk+0.5) / (N-ntk[term]-R+rtk+0.5))
    
    # Calculates the mean of all w5-scores
    # If there exist no relevant documents, the mean is zero, due to the fact
    # that it is not possible to calculate the division by zero.  
    if R == 0:
        meanW5 = 0
    else: 
        for term, rtk in T.items():
            meanW5 = rtk
        meanW5 = meanW5/len(T)
    
    # Feature selection based on the mean weight5-score and the parameter theta
    Features = {t:r for t,r in T.items() if r > meanW5 + theta}

    return Features

In [17]:
def My_PRM(weighting_function, collection: bow_document_collection, query: dict, threshold: int, theta: int) -> dict:
    """
    This function represents a Pseudo-Relevance-Model (PRM) to rank documents 
    based on pseudo feedback. It accepts a weighting function as parameter such
    as the BM25- and Vector Space Model-algorithm to calculate weights for 
    each document in a given collection with respect to a given query. The 
    parameters threshold and theta to fine-tune the algorithm. The PRM returns
    a sorted dictionary.
    """
    
    # Ensure that the weighting function is callable
    if not callable(weighting_function):
        raise TypeError("weighting_function must be a callable function.")
    
    # Initializations
    marked_documents = {}
    documents_scores = {}

    # 1) Based on the given query, PRM calculates bm25-score/VSM-score for each document
    weighting_result = weighting_function(collection, query)

    # 2) Based on a defined threshold (e.g. value=1.0), the algorithm marks relevant documents
    # (positive label 1) that are greater than the defined threshold, otherwise as irrelevant (negative label 0)
    for doc, score in weighting_result.items():
        marked_documents[doc] = 1 if score > threshold else 0

    # 3) Calculates w5-score to identify a set of features
    w5_results = w5(collection=collection, marked_documents=marked_documents, theta=theta)

    # 4) Calculate document scores based on the identified features
    for doc_ID, doc in collection.docs.items():
        documents_scores[doc_ID] = 0
        for term, frequency in doc.get_term_list().items():
            documents_scores[doc_ID] += frequency * w5_results.get(term, 0)
        
    # Return documents in descending order
    return dict(sorted(documents_scores.items(), key=lambda item: item[1], reverse=True))


In [57]:
My_PRM_results = {}

for query_key, col in document_set.items():
    query_param = query_frame.loc[query_frame['number'] == query_key, 'weighted_terms'].iloc[0]
    My_PRM_results[query_key] = My_PRM(weighting_function=BM25, collection=col, query=query_param, threshold=0.7, theta=0)  # NOTE: GSCV threshold/theta?
    write_scores_to_file(My_PRM_results[query_key], f"My_PRM{query_key}Ranking")

# Task 4: Model Testing

**Description:** Use Python to implement three models: `BM25`, `JM_LM`, and `My_PRM`, and **test them on the given 50 data collections for the corresponding 50 queries (topics)**. 

Design Python programs to implement these three models. You can use a .py file (or a .ipynb file) for each model.


For each long query, your python programs will produce ranked results and save them into .dat files. For example, for query R107, you can save the ranked results of three models into “BM25_R107Ranking.dat”, “JM_LM_R107Ranking.dat”, and “My_PRM_R107Ranking.dat”, respectively by using the following format:
- The first column is the document id (the itemid in the corresponding XML document)
- The second column is the document score (or probability).

**Describe:** 
- Python packages or modules (or any open-source software) you used
- The data structures used to represent a single document and a set of documents for each model (you can use different data structures for different models).


You also need to **test the three models on the given 50 data collections for the 50 queries (topics) by *printing out the top 15 documents* for each data collection (in descending order)**. The **output will also be put in the appendix of your final report**.

In [21]:
## bm25 and jm_lm models have already been implemented & made .dat files
#need to implement my_prm model

In [58]:
#note: the outputs of the top-15 for each model goes in the appendix of the final report
##Note: Could have the sorting in the earlier code, before writing it into the file?
def get_top15(model_results):
    """
    Takes the model results, prints out the top-15 sorted by weights

    input:
    model results (dict), in the form of {query number, {documentID, document_score}}

    output:
    Prints top-15 list of documents for each query in descending order
    """

    # NOTE: check for less than 15, prob none but worth evaluating

    #iterating over each set of {query:predictions}, where predictions is a dictionary of {docid : document weight}
    for(query, predictions) in model_results.items():
        print('Query' + str(query) + ' (DocID Weight):')        #printing query number + header for query

        #For the given query, sort the document weights and take the top 15 scores
        sorted_weights_top15 = {k: v for k, v in sorted(predictions.items(), key=lambda item: item[1], reverse=True)[:15]}        #sorting weights for query, selecting the top 15

        #iterating over each docid:weight for the predictions, for the given query
        for (docid, weight) in sorted_weights_top15.items():   #selecting each pairing
            print(docid + ': ' + str(weight))                  #printing the {docid:weight}



In [61]:
BM25_results['104']

{'11083': 0.0,
 '11481': 0,
 '11485': 0,
 '11923': 0,
 '11930': 0,
 '11960': 0,
 '11979': 0,
 '12479': 0,
 '12812': 0,
 '12825': 0.0,
 '13313': 0,
 '13314': 0.0,
 '13320': 0,
 '13332': 0.0,
 '13335': 0,
 '13342': 0,
 '13345': 0,
 '14707': 0,
 '14713': 0.0,
 '15077': 0,
 '15082': 0,
 '15200': 0,
 '16390': 0.0,
 '16897': 0,
 '16899': 0,
 '16922': 0.0,
 '16954': 0,
 '17517': 0,
 '17587': 0.0,
 '17609': 0.0,
 '17619': 0,
 '19033': 0.0,
 '19224': 0.0,
 '19599': 0,
 '19601': 0,
 '19615': 0,
 '19634': 0.0,
 '19781': 0.0,
 '20127': 0,
 '20133': 0,
 '20156': 0.0,
 '20160': 0.0,
 '20199': 0.0,
 '20201': 0.0,
 '20688': 0,
 '20705': 0.0,
 '20766': 0,
 '21895': 0,
 '22690': 0,
 '22751': 0.0,
 '22961': 0.0,
 '23377': 0.0,
 '24485': 0,
 '24490': 0,
 '24494': 0.0,
 '24498': 0.0,
 '24515': 0.0,
 '24550': 0.0,
 '25096': 0,
 '25100': 0,
 '25103': 0.0,
 '25205': 0,
 '25699': 0,
 '25824': 0.0,
 '26080': 0.0,
 '26113': 0.0,
 '26582': 0.0,
 '26598': 0.0,
 '26603': 0.0,
 '26611': 0.0,
 '27384': 0,
 '27386': 0

In [60]:
get_top15(BM25_results)

Query101 (DocID Weight):
46547: 1.8035494303636321
46974: 1.8035494303636321
62325: 1.3241473404899702
6146: 1.0920629621150366
22170: 0.9772324022589438
61329: 0.3267075259817516
61780: 0.2765908986426687
22513: 0.21718684610512737
82330: 0.17502274720055333
39496: 0.15040395022153746
18586: 0.0
26642: 0.0
26847: 0.0
27577: 0.0
30647: 0.0
Query102 (DocID Weight):
73038: 1.9654062144138527
26061: 1.8836675215878933
58476: 1.8505211923545215
57914: 1.827692660880318
78836: 1.621372998437843
76635: 1.5728476262157196
12769: 1.5246285213635489
65414: 1.4816271036527422
12767: 1.480572707240055
82227: 1.425411928703988
25096: 1.3785970499333262
24515: 1.256085766054691
26611: 1.1950064608823971
33172: 1.1008705051298882
29908: 1.0882226835128361
Query103 (DocID Weight):
27426: 1.3516934004670722
27106: 1.2556822622000006
14314: 1.2066512177806743
59459: 1.1649156057662169
83370: 1.1010680630307066
81463: 1.0164095407248623
54533: 0.9569524880588856
20159: 0.9238650166036071
26258: 0.194892

In [24]:
get_top15(JM_LM_results)

Query101 (DocID Weight):
46547: 0.0003686566682181676
46974: 0.0003686566682181676
62325: 5.67909896341436e-05
61329: 6.6829612013916405e-06
6146: 5.343522519648502e-06
61780: 3.12372078516988e-06
22170: 2.8915792887255432e-06
22513: 2.256512809426008e-06
82330: 1.4024817829811885e-06
39496: 1.1054548156956005e-06
18586: 2.972651605231867e-07
26642: 2.972651605231867e-07
26847: 2.972651605231867e-07
27577: 2.972651605231867e-07
30647: 2.972651605231867e-07
Query102 (DocID Weight):
78836: 5.193880353327135e-07
58476: 3.6238288552765587e-07
26061: 1.6922824849415996e-07
57914: 1.6085372175349953e-07
76635: 1.2952021368676935e-07
73038: 9.530510823220392e-08
12769: 8.424995473011488e-08
12767: 6.59522374868332e-08
25096: 3.631100986911571e-08
24515: 2.982648908981551e-08
82227: 2.88192904099521e-08
65414: 1.4717139870081546e-08
26611: 1.4701832095644448e-08
33172: 9.634466041875688e-09
29908: 9.11674417110131e-09
Query103 (DocID Weight):
81463: 5.179167414484945e-07
14314: 4.7799827406195

In [25]:
# get the top 15 from my_prm when implemented
## get_top15(My_PRM_results)

# Task 5: Model Evaluation

**Description:** Use three effectiveness measures to evaluate the three models.

In this task, you need to **use the relevance judgments (EvaluationBenchmark.zip)** to **compare with the ranking outputs in the folder of “RankingOutputs” for the selected effectiveness metric** for the three models.


You need to use the following three different effectiveness measures to evaluate the document ranking results you saved in the folder “RankingOutputs”:
1) Average precision (and MAP)
2) Precision@10 (and their average)
3) Discounted cumulative gain at rank position 10 ($p = 10$), $DCG_{10}$ (and their average):  
    $DCG_p=rel_i+\sum_{i=2}^p\frac{rel_i}{log_2(i)}$  
        $rel_i=1$ if the document at position $i$ is releveant; otherwise, it is 0.

Evaluation results can be summarized in tables or graphs. Examples are provided in the sepcification sheet.

### Getting evaluation benchmark set

In [77]:
BM25_results

{'101': {'46547': 1.8035494303636321,
  '46974': 1.8035494303636321,
  '62325': 1.3241473404899702,
  '6146': 1.0920629621150366,
  '22170': 0.9772324022589438,
  '61329': 0.3267075259817516,
  '61780': 0.2765908986426687,
  '22513': 0.21718684610512737,
  '82330': 0.17502274720055333,
  '39496': 0.15040395022153746,
  '18586': 0.0,
  '26642': 0.0,
  '26847': 0.0,
  '27577': 0.0,
  '30647': 0.0,
  '63261': 0.0,
  '77909': 0.0,
  '80425': 0.0,
  '80950': 0.0,
  '81463': 0.0,
  '82454': 0.0,
  '82912': 0.0,
  '83167': 0.0},
 '102': {'73038': 1.9654062144138527,
  '26061': 1.8836675215878933,
  '58476': 1.8505211923545215,
  '57914': 1.827692660880318,
  '78836': 1.621372998437843,
  '76635': 1.5728476262157196,
  '12769': 1.5246285213635489,
  '65414': 1.4816271036527422,
  '12767': 1.480572707240055,
  '82227': 1.425411928703988,
  '25096': 1.3785970499333262,
  '24515': 1.256085766054691,
  '26611': 1.1950064608823971,
  '33172': 1.1008705051298882,
  '29908': 1.0882226835128361,
  '28

In [79]:
folder_path = 'EvaluationBenchmark'
relevance_scores = evaluation_loader(folder_path)
relevance_scores

{'101': {'6146': 0,
  '18586': 0,
  '22170': 0,
  '22513': 0,
  '26642': 0,
  '26847': 0,
  '27577': 0,
  '30647': 0,
  '39496': 1,
  '46547': 1,
  '46974': 1,
  '61329': 0,
  '61780': 0,
  '62325': 1,
  '63261': 1,
  '77909': 0,
  '80425': 0,
  '80950': 0,
  '81463': 0,
  '82330': 1,
  '82454': 1,
  '82912': 0,
  '83167': 0},
 '102': {'3265': 1,
  '3827': 1,
  '3828': 1,
  '3833': 1,
  '3834': 1,
  '3835': 1,
  '3837': 1,
  '3972': 0,
  '3976': 1,
  '4306': 1,
  '4310': 1,
  '4358': 1,
  '4395': 1,
  '4439': 1,
  '4881': 1,
  '4933': 0,
  '5862': 1,
  '6497': 1,
  '6498': 1,
  '6503': 1,
  '6534': 1,
  '6635': 0,
  '6636': 1,
  '6735': 1,
  '7115': 1,
  '7118': 1,
  '7502': 1,
  '7937': 0,
  '8327': 1,
  '8333': 1,
  '8455': 1,
  '9358': 0,
  '9479': 0,
  '9703': 1,
  '9726': 0,
  '9790': 1,
  '10182': 0,
  '11083': 0,
  '11481': 1,
  '11485': 1,
  '11922': 1,
  '11923': 1,
  '11930': 1,
  '11960': 0,
  '11979': 1,
  '12479': 1,
  '12767': 0,
  '12769': 0,
  '12812': 1,
  '12825': 0,


### Average Precision (MAP)

In [29]:
def calculate_avg_precision(rel_docs, model_results, threshold):

    """"
    This function calculates the mean average precision (MAP) for each topic in a collection

    MAP is calculated by checking whether the document deemed relevant by the model (score exceeds the threshold) is considered relevant by the evaluation benchmark.

    Inputs:
    rel_docs (dict): A dictionary of {topic : {docid:relevance_score}} for the benchmark sets: used as the 'ground truth' scores
    model_results (dict): A dictionary of predictions for doc relevance, in the form of {topic : {docid:relevance_score}} from the model 
    threshold (float): A defined value, where if the model predicts a score above the threshold, the document has predicted the document is 'relevant'

    Returns:
    coll_precisions(dict): A dictionary of {topic : precision} for each topic in the collection
    """

    coll_precisions = {}

    for topic in rel_docs.keys():
        if topic in model_results:

           #getting the inner {docID:score} dictionaries
            rel_dict = rel_docs[topic]        #docID:relevance from benchmark
            model_dict = model_results[topic]  #docID: score from model - retrieved docs

            # Calculate the number of relevant docs
            R = 0
            for value in rel_dict.values():
                if value == 1:
                    R = R + 1


            # Calculate the number of retrieved docs
            R1 = 0
            for value in model_dict.values():
                if value > threshold:
                    R1 = R1 + 1
            

            # Calculate the number of correctly retrived docs (true positives)
            RR1 = 0
            for (docID, value) in model_dict.items():
                if (value > threshold) and (rel_dict[docID] == 1):
                    RR1 = RR1 + 1


            #calculate precision
           
            #if statement to handle any instances where there were no retrieved documents - returns score of 0
            if R1 > 0.0:
                p = float(RR1)/float(R1)
            else:
                p = 0.0


            #storing precision for each topic as {topic : precision}
            ####Note##### Can have R, R1, RR1 here to get the relevant docs, retrieved docs, and correctly retrieved docs if needed for checking outputs.
            coll_precisions[topic] = {
                'precision' : p
            }
        else:
            coll_precisions[topic] = {
                'precision' : 0
            }

    return coll_precisions



#function to calculate the averages
def calculate_map(coll_scores):
    """
    This function calculates the mean average precision, by averaging the precision for each query in a collection

    input:
    collection_scores(dict): A dict of {topic : scores}
    
    output:
    avg_score(float) : an average score for the collection
    """
    total_score = 0.0       #initialise score

    for topic, preds in coll_scores.items():    #iterate over each topic
        for score in preds.values():            #get the {docid:precision}
            total_score += score

    num_scores = len(coll_scores)

    avg_score = total_score / num_scores

    return avg_score


#### MAP for BM_25

In [30]:
# Define the threshold
threshold = 0.25

# Calculate precision for each query
bm25_precision = calculate_avg_precision(relevance_scores, BM25_results, threshold)

# Print the precision for each query
for topic, metric in bm25_precision.items():
    print(f" Topic {topic}: Precision: {metric['precision']}")

 Topic 101: Precision: 0.375
 Topic 102: Precision: 0.7878787878787878
 Topic 103: Precision: 0.1724137931034483
 Topic 104: Precision: 0.0
 Topic 105: Precision: 0.0
 Topic 106: Precision: 0.1875
 Topic 107: Precision: 0.25
 Topic 108: Precision: 0.10526315789473684
 Topic 109: Precision: 0.4
 Topic 110: Precision: 0.3125
 Topic 111: Precision: 0.17647058823529413
 Topic 112: Precision: 0.5454545454545454
 Topic 113: Precision: 0.43478260869565216
 Topic 114: Precision: 0.5
 Topic 115: Precision: 0.16666666666666666
 Topic 116: Precision: 0.25
 Topic 117: Precision: 0.0
 Topic 118: Precision: 0.25
 Topic 119: Precision: 0.0
 Topic 120: Precision: 0.2857142857142857
 Topic 121: Precision: 0.45161290322580644
 Topic 122: Precision: 0.6521739130434783
 Topic 123: Precision: 0.18181818181818182
 Topic 124: Precision: 0.0
 Topic 125: Precision: 0.5833333333333334
 Topic 126: Precision: 0.0
 Topic 127: Precision: 0.4166666666666667
 Topic 128: Precision: 0.09090909090909091
 Topic 129: Prec

In [31]:
# Get the mean average precision for bm_25
calculate_map(bm25_precision)

0.2556163850770885

#### MAP for JM_LM

In [32]:
# Define the threshold
threshold = 0.0001

# Calculate precision for each query
jmlm_precision = calculate_avg_precision(relevance_scores, JM_LM_results, threshold)

# Print the precision for each query
for topic, metric in jmlm_precision.items():
    print(f" Topic {topic}: Precision: {metric['precision']}")


 Topic 101: Precision: 1.0
 Topic 102: Precision: 0.0
 Topic 103: Precision: 0.0
 Topic 104: Precision: 0.0
 Topic 105: Precision: 0.0
 Topic 106: Precision: 0.0
 Topic 107: Precision: 0.0
 Topic 108: Precision: 0.0
 Topic 109: Precision: 0.0
 Topic 110: Precision: 0.0
 Topic 111: Precision: 0.0
 Topic 112: Precision: 0.0
 Topic 113: Precision: 0.0
 Topic 114: Precision: 0.0
 Topic 115: Precision: 0.0
 Topic 116: Precision: 0.0
 Topic 117: Precision: 0.125
 Topic 118: Precision: 0.0
 Topic 119: Precision: 0.0
 Topic 120: Precision: 0.0
 Topic 121: Precision: 0.0
 Topic 122: Precision: 0.0
 Topic 123: Precision: 0.0
 Topic 124: Precision: 0.0
 Topic 125: Precision: 0.7142857142857143
 Topic 126: Precision: 0.6153846153846154
 Topic 127: Precision: 0.0
 Topic 128: Precision: 0.0
 Topic 129: Precision: 0.0
 Topic 130: Precision: 0.0
 Topic 131: Precision: 0.16666666666666666
 Topic 132: Precision: 0.0
 Topic 133: Precision: 0.0
 Topic 134: Precision: 0.0
 Topic 135: Precision: 0.0
 Topic 

In [33]:
#Calculate map for jm_lm
calculate_map(jmlm_precision)

0.06969946719946721

#### MAP for My_PRM

### Precision @ 10

In [34]:
# Functions

def precision_at_k(rel_docs, retrieved_docs, k, threshold):

    """
    This function calculates the precision at k for a given set of relevant and retrieved items.
    It sorts values by the scores, from highest to lowest, taking the top-k scores.
    If the model predicts that the document is relevant (score > threshold), and the document exists in the rel_docs (it is actually relevant), then the score is added. 

    Precision is calculated by (number of retrieved docs / number of retrieved relevant docs)
    
    Inputs:
    rel_docs(dict): a dictionary of {docid:relevance_score} for a given query. This is the 'ground truth' used to check if the document is relevant or not
    retrieved_docs(dict): A dictionary of {docid:relevance_score} for a given query. This is the model predictions 
    k(int) : The rank position up to which precision is calculated
    threshold(float): A threshold to determine relevance. If the model's score for that document exceeds the threshold, the model considers that document relevant

    output:
    precision(float): the precision at rank k for the model

    """
    #sort the model's scores in descending order, taking the top 'k' scores
    sorted_predictions = sorted(retrieved_docs.items(), key=lambda item: item[1], reverse=True)[:k]


    #Make dictionary of {docID : relevant_score} using  rel_docs
    relevant_items = {}     #initialise dictionary

    for (docid, score) in rel_docs.items():     #for each document_id:score in the dict
        if score == 1:              #if the score is equal to or above threshold (1 in this case from eval_benchmark)
            relevant_items[docid] = score       #add relevant documentID, and score to the dictionary

    
    #count number of retrieved docs
    R1 = 0
    for (docid, score) in sorted_predictions:
        if (score >= threshold):
            R1 = R1 + 1

    #calculating the number of documents in the retrieved doc collection that are correctly retrieved, using the top-k predictions
    RR1 = 0      #initialise count 

    for (docid, score) in sorted_predictions:       #for each document in the model predictions
        if docid in relevant_items:                 #if the document retrieved is correctly in the 'relevant documents' dictionary
            if score >= threshold:            #if the score given to the model by the document is equal to or greater than the specified threshold
                RR1 = RR1 + 1                        #increases the count by 1
                

    #calculate precision at k

    #if statement to handle any instances where there were no retrieved documents - returns score of 0
    if R1 > 0.0 and RR1 > 0.0:
        precision = float(RR1) / float(R1)      #calculate precision (number of correctly retrieved docs / number of retrieved docs)
    else:
        precision = 0.0

    return precision


def calculate_precision_at_10(rel_docs, model_results, threshold):
    """
    This function calculates the precision at rank 10 for a collection of topics.
    It does this by passing each {docid:query} for a given topic to the precision_at_k function, and adding each result to a dictionary

    input:
    rel_docs(dict): a dictionary of {topic : {docid:relevance_score}}. This is the 'ground truth' used to check if the document is relevant or not
    model_results(dict): a dictionary of {topic : {docid:relevance_score}}. This is the model predictions, or 'retrieved documents'
    threshold(float): documents with a score/weight that exceeds this threshold are considered 'retrieved docs', ie., documents the model has predicted are relevant.

    output:
    precision_at_10_per_topic(dict) : A dictionary of {topic:p10}, where each topic is a string, and each 'p10' is a float value of its precision at rank 10


    """
    precision_at_10_per_topic = {}      #initialise dictionary

    #iterate over each topic in the dict
    for topic in rel_docs.keys():
        if topic in model_results:    #ensuring the topic exists in the predictions
            #passing to the precision_at_k() function
            precision_at_10 = precision_at_k(rel_docs[topic], model_results[topic], 10, threshold)
            #assigning the value for that topic in the dictionary  
            precision_at_10_per_topic[topic] = precision_at_10
        else:
            precision_at_10_per_topic[topic] = 0.0  #otherwise score = 0
    return precision_at_10_per_topic


In [35]:
#note: this can be used for any {topic:score} dict - so at the moment, both p10 and dcg
def calculate_avg_score(collection_scores):
    """
    This function calculates the average score for a collection of {topic:score}. It sums up each value in the dictionary, then divides by the number of entries

    input: 
    collection_scores(dict): A collection of {ID:scores} 

    output:
    avg_score(float): a value for the average score in the 'values' part of the dictionary
    
    """

    total_score = 0.0 #initialise score
    total_score = sum(collection_scores.values())

    num_scores = len(collection_scores)

    avg_score = total_score / num_scores

    return avg_score 


#### BM_25 Precision @ 10 

In [36]:
# Define the threshold
model_threshold = 0.25

# Calculate precision at 10
bm_precision_at_10_per_topic = calculate_precision_at_10(relevance_scores, BM25_results, model_threshold)

# Print the precision at 10 for each group
for topic, precision in bm_precision_at_10_per_topic.items():
    print(f"Topic {topic}: Precision at 10: {precision:.3f}")


Topic 101: Precision at 10: 0.375
Topic 102: Precision at 10: 0.500
Topic 103: Precision at 10: 0.500
Topic 104: Precision at 10: 0.000
Topic 105: Precision at 10: 0.000
Topic 106: Precision at 10: 0.200
Topic 107: Precision at 10: 0.250
Topic 108: Precision at 10: 0.000
Topic 109: Precision at 10: 0.400
Topic 110: Precision at 10: 0.300
Topic 111: Precision at 10: 0.000
Topic 112: Precision at 10: 0.600
Topic 113: Precision at 10: 0.500
Topic 114: Precision at 10: 0.500
Topic 115: Precision at 10: 0.200
Topic 116: Precision at 10: 0.200
Topic 117: Precision at 10: 0.000
Topic 118: Precision at 10: 0.300
Topic 119: Precision at 10: 0.000
Topic 120: Precision at 10: 0.600
Topic 121: Precision at 10: 0.400
Topic 122: Precision at 10: 0.400
Topic 123: Precision at 10: 0.100
Topic 124: Precision at 10: 0.000
Topic 125: Precision at 10: 0.700
Topic 126: Precision at 10: 0.000
Topic 127: Precision at 10: 0.400
Topic 128: Precision at 10: 0.100
Topic 129: Precision at 10: 0.300
Topic 130: Pre

In [37]:
calculate_avg_score(bm_precision_at_10_per_topic)

0.262

#### JM_LM Precision @ 10

In [38]:
# Define the threshold
threshold = 0.00001

# Calculate precision at 10
jmlm_precision_at_10_per_topic = calculate_precision_at_10(relevance_scores, JM_LM_results,threshold)

# Print the precision at 10 for each group
for topic, precision in jmlm_precision_at_10_per_topic.items():
    print(f"Topic {topic}: Precision at 10: {precision:.3f}")

Topic 101: Precision at 10: 1.000
Topic 102: Precision at 10: 0.000
Topic 103: Precision at 10: 0.000
Topic 104: Precision at 10: 0.000
Topic 105: Precision at 10: 0.000
Topic 106: Precision at 10: 0.000
Topic 107: Precision at 10: 0.000
Topic 108: Precision at 10: 0.000
Topic 109: Precision at 10: 0.000
Topic 110: Precision at 10: 0.000
Topic 111: Precision at 10: 0.000
Topic 112: Precision at 10: 1.000
Topic 113: Precision at 10: 0.000
Topic 114: Precision at 10: 0.000
Topic 115: Precision at 10: 0.000
Topic 116: Precision at 10: 0.100
Topic 117: Precision at 10: 0.200
Topic 118: Precision at 10: 0.000
Topic 119: Precision at 10: 0.000
Topic 120: Precision at 10: 0.000
Topic 121: Precision at 10: 0.000
Topic 122: Precision at 10: 1.000
Topic 123: Precision at 10: 0.000
Topic 124: Precision at 10: 0.400
Topic 125: Precision at 10: 0.700
Topic 126: Precision at 10: 0.600
Topic 127: Precision at 10: 0.000
Topic 128: Precision at 10: 0.000
Topic 129: Precision at 10: 0.000
Topic 130: Pre

In [39]:
calculate_avg_score(jmlm_precision_at_10_per_topic)

0.14422222222222222

In [49]:
query_frame

,number,title,description,narrative,parsed_titles,weighted_terms
0,101,Economic espionage,What is being done to counter economic espiona...,Documents which identify economic espionage ca...,"{'econom': 1, 'espionag': 1}","{'econom': 1.5714285714285714, 'espionag': 0.7..."
1,102,"Convicts, repeat offenders",Search for information pertaining to crimes co...,Relevant documents are those which cite actual...,"{'offend': 1, 'convict': 1, 'repeat': 1}","{'offend': 1, 'convict': 2.0, 'repeat': 1}"
2,103,Ferry Boat sinkings,Documents will report on any sinkings of Ferry...,Documents that identify any instances where a ...,"{'boat': 1, 'sink': 1, 'ferri': 1}","{'boat': -2.2857142857142856, 'sink': 1, 'ferr..."
3,104,Rescue of kidnapped children,Identify a kidnapping of a child or children w...,Documents discussing abducted or kidnapped chi...,"{'kidnap': 1, 'children': 1, 'rescu': 1}","{'kidnap': 2.425, 'children': 2.1, 'rescu': 2...."
4,105,Sport Utility Vehicles U.S.,Find documents that will illustrate the phenom...,Documents that discuss the growth in ownership...,"{'vehicl': 1, 'util': 1, 'sport': 1}","{'vehicl': 4.9375, 'util': 1.8125, 'sport': 3.25}"
5,106,Government supported school vouchers,Research documents on the pros/cons of governm...,Documents containing statements by elected off...,"{'voucher': 1, 'govern': 1, 'support': 1, 'sch...","{'voucher': -2.25, 'govern': -4.5, 'support': ..."
6,107,Tourism Great Britain,Retrieve documents pertaining to tourism into ...,"Documents about Scotland, Wales and only North...","{'great': 1, 'tourism': 1, 'britain': 1}","{'great': 0.6666666666666667, 'tourism': -46.0..."
7,108,Harmful weight-loss drugs,Identify medicines used for obesity or weight-...,"Relevant documents will show specific, harmful...","{'harm': 1, 'loss': 1, 'drug': 1, 'weight': 1}","{'harm': 0.33333333333333337, 'loss': -11.0, '..."
8,109,Child custody cases,Research reports on child custody cases.,Relevant documents concentrate on custody case...,"{'case': 1, 'child': 1, 'custodi': 1}","{'case': 2.55, 'child': 0.44999999999999996, '..."
9,110,Terrorism Middle East tourism,What effect has Middle East terrorism had on t...,Relevant documents directly correlate terroris...,"{'middl': 1, 'east': 1, 'terror': 1, 'tourism'...","{'middl': -17.8, 'east': -36.0, 'terror': -0.6..."


#### My_PRM Precision @ 10

In [40]:
#implement p@10 function for My_PRM model

### Discounted Cumulative Gain at rank position 10

In [41]:
## Functions ##

import math

# Function to calculate dcg at position k

def dcg_at_k(rel_docs, model_predictions, k):
    """
    This function calculates the discounted cumulative rank gain (dcg) at position k for a set of {docid:score} predictions.

    DCG is calculated by summing relevance scores, divided by the log(2) of their rank position.

    For each document in top-k, check whether it is relevant.
    Relevance is determined by whether the document has the score (1) in the relevance_docs set
        if document at position i is relevant, relevance score of 1 is assigned
        if document at position i is irrelevant, score is zero

    Inputs:
    rel_docs (dict): A dictionary with the benchmark relevance for a given query. Stored in the form of {docid : relevance}
    model_predictions (dict): A dictionary of model predictions for a given query (topic). Stored in the form of {docid : relevance}
    k (int): The rank position DCG is calculated for.

    Output:
    dcg (float): Calculated dcg value for position k 

    """

    #sort the model scores based on values, taking only top-k results
    sorted_preds = sorted(model_predictions.items(), key = lambda item : item[1], reverse=True)[:k]

    #initialise dcg counter
    dcg = 0.0

    #calculate dcg for position 1
    #check whether the document at that position is relevant (has a score of 1 in the rel_docs set)
    docid, score = sorted_preds[0]      #get the docid & score for the highest-ranked document
    if rel_docs.get(docid, 0) == 1:     #check if the document is relevant in the benchmark set
        relevance = 1                   #assign a value of 1
        dcg += relevance                #add relevance to the dcg 

    #iterate over the top-k sorted predictions
    for i in range (1, k):              #starting from position 2
        if i < len(sorted_preds):       #ensuring we haven't exceeded the number of predictions
            docid, score = sorted_preds[i]  #get the document ID and score for the current rank
            if rel_docs.get(docid, 0) == 1:     #check whether the score exists in the benchmark set
                relevance = 1
                discounted_rel = relevance / math.log2(i + 1)       #apply the dcg formula by dividing by log i 

                dcg += discounted_rel           #add to running total
    

    #return calculated DCG value at position k
    return dcg


#Function to calculate dcg for a collection of topics

def calculate_dcg10(rel_docs, model_predictions):
    """
    This calculates dcg at position 10 for each topic (query) in a collection of {topic : {docid:score}}
    
    For each query, it passes the {docid:relevance score} for the benchmark set, and the model predictions to the dcg_at_k() function 

    Inputs:
    rel_docs(dict): A dictionary where the keys are the topics, and values are a {docid:score} dictionary from the benchmark set (relevance docs)
    model_predictions(dict): A dictionary where the keys are the topics, and values are a {docid:score} dictionary for predictions from our model (retrieved docs)

    output:
    dcg10_scores(dict): A dictionary where the keys are the topics, and the values are the DCG10 values for the given model
    """

    #initialise a dictionary to store the DCG at 10 score for each topic
    dcg10_scores = {}

    #iterate over each topic in the relevance scores documents
    for topic in rel_docs.keys():

        #check if the topic also exists in the model predictions
        if topic in model_predictions:

            #calculate dcg @ 10 for this topic
            #pass to calculate dcg_at_k, using k = 10 
            dcg_at_10 = dcg_at_k(rel_docs[topic], model_predictions[topic], k = 10)

            #store the results in a dictionary
            dcg10_scores[topic] = dcg_at_10

        else:
            dcg10_scores[topic] = 0.0

    return dcg10_scores


#### BM_25 DCG10

In [42]:
bm25_dcg10 = calculate_dcg10(relevance_scores, BM25_results)

for (topic, score) in bm25_dcg10.items():
    print(f"Topic {topic}: {score:.3f}")

Topic 101: 3.247
Topic 102: 2.749
Topic 103: 2.674
Topic 104: 3.868
Topic 105: 2.232
Topic 106: 1.387
Topic 107: 1.500
Topic 108: 0.000
Topic 109: 1.819
Topic 110: 1.320
Topic 111: 0.000
Topic 112: 2.398
Topic 113: 3.432
Topic 114: 3.018
Topic 115: 1.500
Topic 116: 0.732
Topic 117: 1.133
Topic 118: 1.220
Topic 119: 0.000
Topic 120: 3.189
Topic 121: 2.964
Topic 122: 2.005
Topic 123: 0.500
Topic 124: 1.667
Topic 125: 4.120
Topic 126: 2.948
Topic 127: 2.088
Topic 128: 0.333
Topic 129: 1.931
Topic 130: 1.764
Topic 131: 1.690
Topic 132: 1.631
Topic 133: 3.518
Topic 134: 0.764
Topic 135: 1.421
Topic 136: 0.887
Topic 137: 2.631
Topic 138: 0.315
Topic 139: 1.518
Topic 140: 4.868
Topic 141: 4.754
Topic 142: 0.301
Topic 143: 0.301
Topic 144: 0.688
Topic 145: 0.000
Topic 146: 1.000
Topic 147: 1.174
Topic 148: 0.000
Topic 149: 0.301
Topic 150: 1.877


In [43]:
#calculate dcg average for bm_25 
calculate_avg_score(bm25_dcg10)

1.7475467002197471

#### JM_LM DCG10

In [44]:
jmlm_dcg10 = calculate_dcg10(relevance_scores, JM_LM_results)

for (topic, score) in jmlm_dcg10.items():
    print(f"Topic {topic}: {score:.3f}")

Topic 101: 3.247
Topic 102: 3.434
Topic 103: 1.818
Topic 104: 4.567
Topic 105: 2.754
Topic 106: 2.315
Topic 107: 1.431
Topic 108: 0.688
Topic 109: 4.196
Topic 110: 2.377
Topic 111: 0.000
Topic 112: 3.918
Topic 113: 3.597
Topic 114: 3.395
Topic 115: 1.062
Topic 116: 1.000
Topic 117: 1.301
Topic 118: 1.018
Topic 119: 0.000
Topic 120: 2.918
Topic 121: 4.134
Topic 122: 2.475
Topic 123: 1.000
Topic 124: 2.856
Topic 125: 3.453
Topic 126: 2.860
Topic 127: 2.392
Topic 128: 1.000
Topic 129: 3.178
Topic 130: 1.856
Topic 131: 1.301
Topic 132: 1.431
Topic 133: 3.562
Topic 134: 0.631
Topic 135: 3.059
Topic 136: 2.319
Topic 137: 2.631
Topic 138: 0.657
Topic 139: 2.000
Topic 140: 4.254
Topic 141: 3.820
Topic 142: 0.315
Topic 143: 0.000
Topic 144: 2.178
Topic 145: 0.000
Topic 146: 1.000
Topic 147: 2.377
Topic 148: 0.634
Topic 149: 0.616
Topic 150: 1.818


In [45]:
#calculate dcg average for jm_lm 
calculate_avg_score(jmlm_dcg10)

2.0968713213581576

#### My_PRM DCG10

In [46]:
# Implement DCG10 for My_PRM model

# Task 6: Recommendation

**Description:** Recommend a model based on significance test and your analysis. 

You need to conduct a significance test to compare models. You can choose a t-test to perform a significance test on the evaluation results (e.g., in Tables 1, 2 and 3). 

You can compare models between:
- **BM25** and **JM_LM**
- **BM25** and **My_PRM**
- **JM_LM** and **My_PRM**

Based on $t$-test results ($p$-value and $t$-statistic), you can recommend a model (You ***want the proposed "My_RPM" to be the best because it is your own model***). You can perform the $t$-test using a single effectiveness measure or multiple measures. Generally, using more effectiveness measures provides stronger evidence against the null hypothesis. Note that if the $t$-test is unsatisfactory, you can use the evaluation results to refine **My_PRM** mode. For example, you can adjust parameter settings or update your design and implementation.